# Module 3: Generative Adversarial Network (GAN) Practice

## Practice: Generating Handwritten Digits with GANs

In this notebook, we will train a Generative Adversarial Network (GAN) to generate images of handwritten digits similar to those in the MNIST dataset.

GANs are powerful for generating new content based on real data. The **Generator** network creates digit images that look real, while the **Discriminator** network tries to distinguish between real and fake digits. Over time, the Generator improves until it can fool the Discriminator.

This technique is widely used for deepfake creation, digital art, and image enhancement.


## 1. Import Required Libraries & Load MNIST Dataset

We import TensorFlow/Keras for building models, NumPy for numerical operations, and Matplotlib for visualization.  
We load the MNIST dataset from Keras, normalize pixel values to [-1, 1], and reshape the data for our models.


Nesta celda importamos as librarías necesarias:

- **tensorflow** e **keras**: Para definir modelos e capas.
- **numpy**: Para xerar ruído e manipular arrays.
- **matplotlib.pyplot**: Para visualizar as imaxes.

A continuación cargaremos o dataset MNIST.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

# Load and prepare the MNIST dataset
(train_images, _), (_, _) = keras.datasets.mnist.load_data()
train_images = (train_images.astype('float32') - 127.5) / 127.5  # Normalize to [-1,1]
train_images = np.expand_dims(train_images, axis=-1)  # Add channel dimension

print("Train images shape:", train_images.shape)

## 2. Build the Generator

The Generator network takes a random noise vector (`latent_dim`) as input and outputs a 28×28 grayscale image using dense layers and reshaping.


Nesta celda cargamos e preprocesamos o dataset MNIST:

1. Usamos `keras.datasets.mnist.load_data()` para obter imaxes de díxitos.
2. Normalizamos os píxeles ao rango [-1,1] para axustalos ao rango da saída tanh.
3. Remodelamos os datos para ter forma (n_batches, 28, 28, 1).

Xeramos tamén un lote de ruído para alimentar o xerador no adestramento.

In [ ]:
latent_dim = 100

def build_generator():
    model = keras.Sequential([
        keras.Input(shape=(latent_dim,)),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(28 * 28 * 1, activation='tanh'),
        layers.Reshape((28, 28, 1))
    ])
    return model

generator = build_generator()
generator.summary()

## 3. Build the Discriminator

The Discriminator network takes a 28×28 image as input, flattens it, and uses dense layers to output a probability that the image is real.


Nesta celda definimos o xerador:

- Creamos un modelo **Sequential**.
- Engadimos unha capa **Dense** para transformar o ruído nun vector de tamaño 7×7×128.
- Aplicamos **LeakyReLU** e **BatchNormalization** para estabilizar o adestramento.
- Remodelamos o vector nun tensor 7×7×128.
- Empregamos dúas capas **Conv2DTranspose** sucesivas para chegar a 28×28×1.
- A capa final usa **activation='tanh'** para producir píxeles en [-1,1].

In [ ]:
def build_discriminator():
    model = keras.Sequential([
        keras.Input(shape=(28, 28, 1)),
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=keras.optimizers.Adam(0.0002), loss='binary_crossentropy')
    return model

discriminator = build_discriminator()
discriminator.summary()

## 4. Configure the GAN

We freeze the Discriminator when training the Generator, then compile the combined GAN model.


Nesta celda definimos o discriminador:

- Creamos un modelo **Sequential**.
- Engadimos varias capas **Conv2D** con activación **LeakyReLU** e **Dropout**.
- Estas capas extraen características das imaxes de 28×28×1.
- Achátanse as características co **Flatten**.
- Remátase cunha capa **Dense(1)** con **sigmoid** para indicar real/falso.
- Compilamos o discriminador con optimizador **Adam** e perda **binary_crossentropy**.

In [ ]:
# Combine Generator and Discriminator
discriminator.trainable = False

gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.Model(gan_input, gan_output)

gan.compile(optimizer=keras.optimizers.Adam(0.0002), loss='binary_crossentropy')
gan.summary()

## 5. Train the GAN

We train for a number of epochs. In each step:
1. Sample real images and label them as real (1).
2. Generate fake images and label them as fake (0).
3. Train Discriminator on both real and fake.
4. Train Generator via the GAN model with misleading labels (1) to fool the Discriminator.


Nesta celda implementamos a función de adestramento:

- Percorremos épocas e lotes de datos reais.
- Xeramos un lote de ruído e producimos imaxes falsas co xerador.
- Combinamos imaxes reais e falsas e etiquetas.
- Actualizamos primeiro o discriminador coas súas perdas, logo o xerador coas súas.
- Cada certas épocas, gardamos e mostramos mostras xeradas.

In [ ]:
# Training parameters
epochs = 3000
batch_size = 128
buffer_size = train_images.shape[0]

# Create dataset pipeline
dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(buffer_size).batch(batch_size)

# Training loop
for epoch in range(epochs):
    for real_images in dataset:
        # Train Discriminator
        noise = tf.random.normal([batch_size, latent_dim])
        fake_images = generator(noise, training=True)
        
        real_labels = tf.ones((batch_size, 1))
        fake_labels = tf.zeros((batch_size, 1))
        
        d_loss_real = discriminator.train_on_batch(real_images, real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_images, fake_labels)
        d_loss = 0.5 * (d_loss_real + d_loss_fake)
        
        # Train Generator
        noise = tf.random.normal([batch_size, latent_dim])
        misleading_labels = tf.ones((batch_size, 1))
        g_loss = gan.train_on_batch(noise, misleading_labels)
    
    # Log progress
    if epoch % 500 == 0:
        print(f'Epoch {epoch}, D Loss: {d_loss:.4f}, G Loss: {g_loss:.4f}')

## 6. Generate AI-Created Digits

After training, we generate new digit images by sampling random noise vectors and using the Generator.


Nesta celda executamos o adestramento do GAN:

- Chamamos á función `train` indicando número de épocas e tamaño de lote.
- Ao rematar, visualizamos as imaxes xeradas finais.

Observaremos como, ao longo do adestramento, os díxitos van acadando maior nitidez e realismo.

In [ ]:
# Generate and plot digits
noise = tf.random.normal([5, latent_dim])
generated_images = generator.predict(noise)

plt.figure(figsize=(10, 2))
for i in range(5):
    plt.subplot(1, 5, i + 1)
    plt.imshow(generated_images[i, :, :, 0], cmap='gray')
    plt.axis('off')
plt.show()